In [48]:
schema_ = {
    "properties": {
            "reasoning_2": {
            "description": "Gie a rating between 500 and 10000!",
            "title": "Score",
            "type": "integer",
        },
        "zeasoning": {
            "description": "Why did you give this rating?",
            "title": "Reasoning",
            "type": "string",
        },

        "school_yourself": {
            "description": "Shool yourself. Why is your answer bad?",
            "title": "School Yourself",
            "type": "string",
        },
    },
    "required": ["reasoning_2", "zeasoning", "school_yourself"],
    "propertyOrdering": ["reasoning_2", "zeasoning", "school_yourself"],
    "title": "Test",
    "type": "object",
    "strict": True,
}

In [10]:
from google import genai
from google.genai import types
import base64
from pydantic import BaseModel, Field
from icecream import ic
from pathlib import Path
from langchain_core.utils.json_schema import dereference_refs
from inputs.prompts.v11.data_models import (
    HolisticEmotionAnalysis,
    add_property_ordering_single_class,
    add_specific_property_ordering,
)
from langsmith import traceable
from dotenv import load_dotenv

load_dotenv()


class Test(BaseModel):
    reasoning: str = Field(description="Why did you give this rating?")
    score: int = Field(
        description="Write a rating of REWE from 500-1000!",
    )
    school_yourself: str = Field(description="Shool yourself. Why is your answer bad?")
#ic(Test.model_json_schema())

In [23]:
prompts_version = "v11"

def read_prompt(filename: str) -> str:
    """Load system prompt from file."""
    folder = Path(f"./inputs/prompts/{prompts_version}")
    return (folder / filename).read_text()

def read_context_sphere(filename: str) -> str:
    """Load system prompt from file."""
    folder = Path(f"./inputs/sp0/")
    return (folder / filename).read_text()


client = genai.Client(
    vertexai=True,
    project="rd-ri-genai-dev-2352",
    #project="rd-smrk-prod518-dev",
    location="us-central1",
)

role_setting_prompt = read_prompt("LFB_role_setting_prompt.md")
role_feedback_prompt = read_prompt("LFB_role_feedback_prompt.md")
user_task_prompt = read_prompt("user_task_prompt.md")

context = read_context_sphere("user_177819_comments_9275_tokens.md")

user_task_prompt_with_context = user_task_prompt.format(
        context_sphere=context
    )



messages = [
    types.Content(role="user", parts = [types.Part.from_text(role_setting_prompt)]),
    types.Content(role="model", parts = [types.Part.from_text(role_feedback_prompt)]),
    types.Content(role="user", parts = [types.Part.from_text(user_task_prompt_with_context)]),

    ]

def generate():


    
    response_schema = dereference_refs(HolisticEmotionAnalysis.model_json_schema())
    response_schema.pop("$defs", None)
    response_schema_properties_ordered = add_specific_property_ordering(response_schema)
    
    print(response_schema)
    

    model = "gemini-2.0-flash-exp"
    contents = messages
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=response_schema_properties_ordered,
        response_modalities=["TEXT"],
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(
                category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"
            ),
            types.SafetySetting(
                category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"
            ),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
    )

    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )
    return response

In [24]:
response = generate()
print(response.text)

{'properties': {'core_affect_analysis': {'description': 'Do not be lazy while working on your tasks!', 'properties': {'thought_process': {'description': "Provide a detailed step-by-step thought process of how you intend to analyze the core affect, *including specific quotes and examples from the user's comments to illustrate your points*, considering both valence (pleasantness) and arousal (activation), and noting any emotional dynamics or changes over time. Reference specific expressions, language, and contextual factors. **Consider how insights from other disciplines like sociology or neuroscience might inform your analysis.**", 'title': 'Thought Process', 'type': 'string'}, 'valence': {'description': "Classify the valence of the user's emotional state, noting any fluctuations. *Cite specific comments or phrases that indicate the valence.*", 'title': 'Valence', 'type': 'string'}, 'arousal': {'description': "Classify the arousal level of the user's emotional state, indicating activati

In [25]:
prompt = read_prompt("step_2_summary_prompt.md")
prompt = prompt.format(classificaton=response.text)

messages_2 = [
    types.Content(role="user", parts = [types.Part.from_text(role_setting_prompt)]),
    types.Content(role="model", parts = [types.Part.from_text(role_feedback_prompt)]),
    types.Content(role="user", parts = [types.Part.from_text(user_task_prompt_with_context)]),
    types.Content(role="model", parts = [types.Part.from_text(response.text)]),
    types.Content(role="user", parts = [types.Part.from_text(prompt)]),
    ]

In [26]:
@traceable(name="Step2:Flash2.0", run_type="llm")
def generate_step2(messages):
    model = "gemini-2.0-flash-exp"
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        max_output_tokens=8192,
        response_modalities=["TEXT"],
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(
                category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"
            ),
            types.SafetySetting(
                category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"
            ),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
        ],
    )

    response = client.models.generate_content(
        model=model,
        contents=messages,
        config=generate_content_config,
    )
    return response

In [29]:
response_2 = generate_step2(messages_2)

In [30]:
ic(response_2.text)


with open("response_2.md", "w") as f:
    f.write(response_2.text)

ic| response_2.text: ('**Inner Monologue**:
                     '
                      'Okay, the user wants a synthesis of the previous analysis. I need to pull '
                      'the key elements from each section and present them in a concise but '
                      'detailed way, making sure to use psychological terminology correctly and '
                      'linking it back to my theory of constructed emotions. The user is requesting '
                      'a summary, but also a level of detail that is not too abstract, so specific '
                      "examples from Pezibua's comments are required. Let's go.
                     "
                      '
                     '
                      '**Synthesized Report**:
                     '
                      '
                     '
                      '**I. Identifying Information:**
                     '
                      '
                     '
                      '*   **Username:** Pezibu